In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'Germany'

nFeatures = 1

nDaysMin = 15
k = 5

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto05252020_forPublication')


confirmedFilename = 'time_series_covid19_confirmed_global.csv'
deathsFilename = 'time_series_covid19_deaths_global.csv'
recoveredFilename = 'time_series_covid19_recovered_global.csv'

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, k):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix + k >= len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+k]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanAbsolutePercentageError_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
            
    # Get MAPE for each day in a list
    MAPEList = []
    for i in yTrueForDayK.keys():
        MAPEList.append(meanAbsolutePercentageError(yTrueForDayK[i], yPredForDayK[i]))
    return np.mean(MAPEList)

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

def medianSymmetricAccuracy_kDay(yTrueListList, yPredListList):
    # Store true and predictions for day 1 in a list, day 2 in a list and so on
    # Keep each list of these lists in a respective dict with key as day #
    yTrueForDayK = {}
    yPredForDayK = {}
    for i in range(len(yTrueListList[0])):
        yTrueForDayK[i] = []
        yPredForDayK[i] = []
    for yTrueList, yPredList in zip(yTrueListList, yPredListList):
        for i in range(len(yTrueList)):
            yTrueForDayK[i].append(yTrueList[i])
            yPredForDayK[i].append(yPredList[i])
    # Get MdSA for each day in a list
    MdSAList = []
    for i in yTrueForDayK.keys():
        MdSAList.append(medianSymmetricAccuracy(yTrueForDayK[i], yPredForDayK[i]))
    return(np.mean(MdSAList))

In [4]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [5]:
# Load all 3 csv files
covidFrDict = {}
covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(os.path.join(dataDir, recoveredFilename))

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin, k)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

Total size of data points for LSTM: 66
Size of training set: 46
Size of validation set: 10
Size of test set: 10


# Vanilla LSTM

In [6]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# nNeurons = 100
# # define model
# model = Sequential()
# model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestNew = XTest.copy()
# for day in range(k):
#     print('Day:', day)
#     yPred = model.predict(XTestNew, verbose=0)
#     for i in range(len(yPred)):
#         yPredListList[i].append(yPred[i][0])
#     print('Prediction:', yPred)
#     XTestNew = np.delete(XTestNew, 0, axis=1)
#     yPred = np.expand_dims(yPred, 2)
#     XTestNew = np.append(XTestNew, yPred, axis=1)

In [7]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 4.016967082464095
Updating best MAPE to 4.016967082464095...
Updating best seed to 0...
1 nan
2 209.62736609787436
3 14.134043650215418
4 4.863668605607055
5 99.98796618231299
6 8.197309484951257
7 2.407146785571937
Updating best MAPE to 2.407146785571937...
Updating best seed to 7...
8 168.04262855466484
9 10.034877213612795
10 6.30017224033711
11 18.87774799401256
12 99.97005410440542
13 9.919487670194636
14 1.290696137848934
Updating best MAPE to 1.290696137848934...
Updating best seed to 14...
15 4.238964318989372
16 99.97959592834533
17 7.3496434769944745
18 3.7967589814624274
19 nan
20 1151.9005717453783
21 2.7330001874456764
22 3.851063394828509
23 4.027932559478022
24 7.2257480930902345
25 8.250439109870602
26 6.860880386413228
27 nan
28 9.970822494789243
29 nan
30 2.1241271072986843
31 7.494053498085281
32 7.9236303968177735
33 4.8432070167175425
34 5.66936429294015
35 9.049924296141501
36 32.17517451678427
37 3.5361671654527895
38 7.059109050448643
39 99.91133448818664
40 9

# Stacked LSTM

In [8]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 5.7101430234227575
Updating best MAPE to 5.7101430234227575...
Updating best seed to 0...
1 8.003062872140585
2 8.155787902928182
3 187.54615877169633
4 1.2977165251595097
Updating best MAPE to 1.2977165251595097...
Updating best seed to 4...
5 509.53167967838255
6 1.1047517947644585
Updating best MAPE to 1.1047517947644585...
Updating best seed to 6...
7 38.2682931154598
8 1.2500502227086625
9 1.6724452704632697
10 99.9984346861755
11 4.851528509574903
12 11.16031260802156
13 99.96179628033707
14 12.999181320762883
15 100.32164503841034
16 99.998146938337
17 915.7398043329501
18 16848.282744801883
19 91.72070646869844
20 99.97937952187667
21 17.729745701549636
22 99.95232448398062
23 5.862430161180605
24 8.234401686192033
25 14.398663078351792
26 99.96050894541295
27 99.95629387353064
28 99.89683024022797
29 2.4128147106956552
30 3.5351854426766467
31 8.100428250829712
32 99.98362857510257
33 99.9868440890786
34 0.6026129573446395
Updating best MAPE to 0.6026129573446395...
Updating

# Bidirectional LSTM

In [9]:
from tensorflow.keras.layers import Bidirectional

# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidNew = XValid.copy()
    for day in range(k):
        yPred = model.predict(XValidNew, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidNew = np.delete(XValidNew, 0, axis=1)
        yPred = np.expand_dims(yPred, 2)
        XValidNew = np.append(XValidNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestNew = XTest.copy()
for day in range(k):
    yPred = model.predict(XTestNew, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestNew = np.delete(XTestNew, 0, axis=1)
    yPred = np.expand_dims(yPred, 2)
    XTestNew = np.append(XTestNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 0.9327214554061818
Updating best MAPE to 0.9327214554061818...
Updating best seed to 0...
1 7.9845547133017165
2 33.785372806190374
3 0.6828934478729518
Updating best MAPE to 0.6828934478729518...
Updating best seed to 3...
4 4.289817697682956
5 5.418052158111868
6 18.901721213202244
7 2.2561073256172746
8 99.92101161827698
9 32.222733805878434
10 13.381991205476254
11 3.3808606080572767
12 1.757854761817012
13 nan
14 0.3100036724952305
Updating best MAPE to 0.3100036724952305...
Updating best seed to 14...
15 3.8062554346098416
16 6.022474262925952
17 221.98173346952973
18 1.9312621133954149
19 3.709398854855192
20 99.88682154698128
21 5.7982481702154445
22 24.825924348981907
23 24.88157647457895
24 2.7151480618008668
25 2.6967308698314874
26 2.281769107969514
27 99.92724681988605
28 7.020285834794413
29 5.055279573186879
30 6.116758388883871
31 5.4750717897697765
32 8.065192380283118
33 0.6830388149079897
34 3.108577195435778
35 99.97654767801042
36 4.799195222427516
37 2.853610451

# CNN LSTM

In [10]:
# import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import Flatten
# from tensorflow.compat.v1.keras.layers import TimeDistributed
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import MaxPooling1D

# nSeq = 5
# nSteps = 3
# nNeurons = 50
# nFeatures = 1
# nFilters = 64

# XTestListNew = XTestList.copy()
# print(XTestListNew.shape)

# model = Sequential()
# model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
# model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# model.add(TimeDistributed(Flatten()))
# model.add(LSTM(nNeurons, activation='relu'))
# model.add(Dense(1))
# opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss='mse')

# XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
# yPred = model.predict(XTestCNN, verbose=0)

# XTestListNew = np.delete(XTestListNew, 0, axis=1)
# XTestListNew = np.append(XTestListNew, yPred, axis=1)

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 5
nSteps = 3

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(101, 201):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)
    
    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidCNN = XValidListNew.reshape((XValidListNew.shape[0], nSeq, nSteps, nFeatures))
        yPred = model.predict(XValidCNN, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestCNN = XTestListNew.reshape((XTestListNew.shape[0], nSeq, nSteps, nFeatures))
    yPred = model.predict(XTestCNN, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

101 3.5847760571644414
Updating best MAPE to 3.5847760571644414...
Updating best seed to 101...
102 12.298788669421379
103 13.369303316250349
104 11.45839398527593
105 4.739277723383199
106 8.474799439599447
107 53.42413250579261
108 5.507665369023659
109 40.635856882705404
110 7.009227127100738
111 49.136853488100066
112 5.771581807164047
113 1.1966741272560946
Updating best MAPE to 1.1966741272560946...
Updating best seed to 113...
114 14.863908878782212
115 6.409056423313925
116 1.517963511340558
117 42.985818578342204
118 4.079409471505
119 7.939538348589595
120 40.81329359473176
121 1.624780477531862
122 7.114770343061217
123 5.9800610394770235
124 38.38563739032588
125 18.374381583788665
126 1.6949189548764694
127 5.730689626227485
128 32.46539599555642
129 3.039229361149664
130 40.54588293698521
131 3.13646517166023
132 43.60012217804042
133 15.406016869376597
134 37.71093646996907
135 43.78040642929223
136 44.72500614846635
137 13.416530526260226
138 41.251349260915205
139 48.4

# ConvLSTM

In [12]:
from tensorflow.keras.layers import ConvLSTM2D

# Number of subsequences to break X into (we do 15 = 5x3, 5 subsequences of size 3 each)
nSeq = 5
nSteps = 3
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
# XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
# XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')
    
    # fit model
    model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

    yPredListList = []
    for day in range(nTest):
        yPredListList.append([])
    XValidListNew = XValidList.copy()
    for day in range(k):
        XValidConv = XValidListNew.reshape((XValidListNew.shape[0], nSeq, 1, nSteps, nFeatures))
        yPred = model.predict(XValidConv, verbose=0)
        for i in range(len(yPred)):
            yPredListList[i].append(yPred[i][0])
        XValidListNew = np.delete(XValidListNew, 0, axis=1)
        XValidListNew = np.append(XValidListNew, yPred, axis=1)

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError_kDay(yValid, yPredListList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain[:,0], epochs=1000, verbose=0)

yPredListList = []
for day in range(nTest):
    yPredListList.append([])
XTestListNew = XTestList.copy()
for day in range(k):
    XTestConv = XTestListNew.reshape((XTestListNew.shape[0], nSeq, 1, nSteps, nFeatures))
    yPred = model.predict(XTestConv, verbose=0)
    for i in range(len(yPred)):
        yPredListList[i].append(yPred[i][0])
    XTestListNew = np.delete(XTestListNew, 0, axis=1)
    XTestListNew = np.append(XTestListNew, yPred, axis=1)
    
MAPE = meanAbsolutePercentageError_kDay(yTest, yPredListList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy_kDay(yTest, yPredListList)
print('Test MdSA:', MdSA)

0 99.88143993398822
Updating best MAPE to 99.88143993398822...
Updating best seed to 0...
1 3.82225222453836
Updating best MAPE to 3.82225222453836...
Updating best seed to 1...
2 1.7252183012393971
Updating best MAPE to 1.7252183012393971...
Updating best seed to 2...
3 2.3950780675828627
4 3.350036456432446
5 2.4707825464363875
6 132.8012576122966
7 91.1532792490698
8 133.4695528103208
9 1.8163359431647361
10 2.488959657270753
11 1.8528827223614286
12 99.88280380390849
13 10.40135445240343
14 7.401147628592762
15 1.3772070688725837
Updating best MAPE to 1.3772070688725837...
Updating best seed to 15...
16 3.3896925465052687
17 0.8607708492387829
Updating best MAPE to 0.8607708492387829...
Updating best seed to 17...
18 129.2578595635709
19 5.0572487067677745
20 4.496376919363793
21 0.6545880595577402
Updating best MAPE to 0.6545880595577402...
Updating best seed to 21...
22 1.159899782792352
23 1.7139730697514768
24 2.9328983347195945
25 3.457278523183716
26 6.593566909317518
27 4.55